In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfdevice import PDFDevice, TagExtractor
import io
import os 
from functools import partial
import multiprocessing
from multiprocessing import Pool 


In [ ]:
def convert(case,fname, pages=None):
    if not pages:
        pagenums = set();
    else:
        pagenums = set(pages);      
    codec = 'utf-8'
    caching = True
    #laparams = LAParams()
    laparams = None
    #password = ''
    #maxpages = 0
    manager = PDFResourceManager(caching=caching) 
    if case == 'txt' :
        output = io.StringIO()
        converter = TextConverter(manager, output, codec=codec, laparams=laparams)     
    if case == 'HTML' :
        output = io.BytesIO()
        converter = HTMLConverter(manager, output, codec=codec, laparams=laparams)
        
    interpreter = PDFPageInterpreter(manager, converter)   
    infile = open(fname, 'rb')

    for index,page in enumerate(PDFPage.get_pages(infile, pagenums,caching=caching, check_extractable=True)):
        interpreter.process_page(page)

    convertedPDF = output.getvalue()


    infile.close();
    converter.close();
    output.close()
    
    return convertedPDF

In [ ]:
def check_pdf(f):
    filename, file_extension = os.path.splitext(f)
    if file_extension.lower() == '.pdf':
        return filename
    else:
        return None
    
def check_make_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)        # If not create the directory, inside their home directory
        return True

In [ ]:
#in_files = in_files[:10]

def process_each_file(file):
    index,f = file
    in_file = os.path.join(in_path,f+'.pdf')
    out_file = os.path.join(out_path,f+'.txt')
    case = "txt"
    try:
        convertedPDF = convert(case, in_file)
        fileConverted = open(out_file, "w",encoding="utf-8")
        fileConverted.write(convertedPDF)
        fileConverted.close()
    except:
        print("failed: ",f)
    
    if int(index)%100 == 0: print('success:   ----   ',index)
    #print("success: ",f)

In [ ]:
if __name__ == '__main__':
    test = False
    ## read file names 
    in_path = '../Documents/'
    out_path = '../txt_files_finished/'
    in_files = os.listdir(in_path)
    
    ## check file exists
    if len(in_files) == 0:
           raise ValueError('could not find input files')
    check_make_dir(out_path)
    
    ## read files
    in_files = [check_pdf(f) for f in in_files]
    in_files = [f for f in in_files if f is not None]
    
    ## see if we want to run a test first
    if test == True: in_files = in_files[:100]
        
    ## multi processes it 
    num_cores = multiprocessing.cpu_count()
    #multi process unpacking
    p = Pool(num_cores)
    partial_unpack = partial(process_each_file)
    process_mp = p.map(partial_unpack,enumerate(in_files))
    p.close()
    p.join()
    print('finish')